----

<html>
<center>
    <font size=6><b> Konkurs DataWorkshop </b></font> <br> 
    w ramach 10 edycji kursu "DS & ML w praktyce" <br><br>
    <b> Przewidywanie emisji CO </b> <br>
     3. miejsce <br> <br>
    <b> Adrian Skutnik </b>
            </center>
</html>

----

## 🗂️ Wczytujemy biblioteki i dane

In [1]:
import numpy as np
import pandas as pd
import utils as u

import xgboost as xgb
import catboost as ctb

%load_ext autoreload
%autoreload 2

In [2]:
df_all = u.prepare_data()
df_all = u.feature_engeenering(df_all)
df_train, df_test = u.split_data(df_all)

X = u.get_X(df_train)
y = u.get_y(df_train)

## Eksperymenty

1. Różne modele drzewiaste na niewielkich parametrach (`max_depth=10, n_estimators=200`)    
    * Wybór padł na CatBoostRegressor (dawał najlepsze wyniki w porównaniach + stabilność z poprzedniego notebooka)


2. Porównanie targetu i `log(target)`. 
    * `Log(target)` sprawdził się dużo lepiej (0.55 vs 0.50 w pierwszych eksperymentach)


3. Różne zestawy cech (od pełnej listy, do selekcji najmocniejszych)
    * Finalny model zbudowany na wybranych cechach (odrzucone cechy kategoryczne i "dzielenie")

In [5]:
# Wszystkie dostępne zmienne, zwykły target
u.run(df_train, plot_lc=False, ylim=None, target_log=False)

In [6]:
# u.run(df_train, plot_lc=False, ylim=None, target_log=True)

In [ ]:
feats = ['TIT', 'TEY', 'TAT', 'AFDP', 'AT + NOX', 'TIT - TEY', 'NOX','GTEP', 'AH', 'TAT - TIT', 'AT', 'AP', 'TIT / TEY', 'CDP', 'CO']
# u.run(df_train[feats], plot_lc=False, ylim=None, target_log=True)

In [ ]:
feats = ['TIT', 'TEY', 'TAT', 'AFDP', 'AT + NOX', 'TIT - TEY', 'NOX','GTEP', 
         'AH', 'TAT - TIT', 'AT', 'AP', 'TIT / TEY', 'CDP', 'CO', 'log_TAT', 'log_TIT', 'sqrt_AFDP+NOX']
# u.run(df_train[feats], plot_lc=False, ylim=None, target_log=True)
# [lgb-10md-200tr]: 0.5092658955903344 +/-0.02186049941165041

In [ ]:
feats = ['TIT', 'TEY', 'TAT', 'AFDP', 'AT + NOX', 'TIT - TEY', 'NOX','GTEP',
         'AH', 'TAT - TIT', 'AT', 'AP', 'CO', 'log_TAT', 'log_TIT', 'AFDP_NOX']
# u.run(df_train[feats], plot_lc=True, ylim=None, target_log=True)
# [lgb-10md-200tr]: 0.4570740010048957 +/-0.023046599035362648

## Budowa finalnego modelu i zapis

In [ ]:
ctb_params = {'n_estimators': 1100.0, 
              'l2_leaf_reg': 2.0391558389346702, 
              'learning_rate': 0.050701938569694155, 
              'max_depth': 15.0
             }

model = ctb.CatBoostRegressor(**ctb_params)

y_pred = u.fit_predict_model(model, df_train[u.FEATS], df_test[u.FEATS], target_log=True) 
df_test.loc[:, 'CO'] = y_pred

In [ ]:
# model = xgb.XGBRegressor(max_depth=10, n_estimators = 500)
# y_pred = u.fit_predict_model(model, df_train, df_test, target_log=True) 
# df_test.loc[:, 'CO'] = y_pred

In [ ]:
# !mkdir -p ../output

## ✔️ Zapisujemy wynik do .csv

In [10]:
df_test[ ["id", "CO"] ].to_csv("../output/220403-2110-ctb-15md-1100tr-hp.csv", index=False)